<a href="https://colab.research.google.com/github/jimin-woo/jimin-woo.github.io/blob/main/Hackathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 필요한 모듈 import 

from glob import glob
import re as re
import os as os
import json as _json
import os.path as path
import sys


import soundfile as sf
import utils as utils
import random as _random


import tensorflow as tf


from random import shuffle as shuffle
from importlib import import_module as import_module
from functools import wraps
from python_speech_features import mfcc
import time
import scipy.io.wavfile as wav
import numpy as np
from six.moves import xrange as range

from abc import ABC, abstractmethod

import pandas as pd
import random as random
import tensorflow as tf
from tqdm import tqdm


import pickle as _pickle
import os.path as _path
from random import shuffle as _shuffle
from importlib import import_module as _import_module
import numpy as _np

import librosa
# 너무 많이 발생하는 경고들을 제거
import sys
import warnings

import os
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"

from pykospacing import spacing
import numpy as np
import re
from pykospacing import spacing
from hanspell import spell_checker


In [ ]:
# 맞춤법, 띄어쓰기, 특수기호 정제를 위한 설치

!pip install git+https://github.com/haven-jeon/PyKoSpacing.git
!pip install git+https://github.com/ssut/py-hanspell.git

In [ ]:
# 텍스트 전처리 함수

import re



INITIALS = list("ㄱㄲㄴㄷㄸㄹㅁㅂㅃㅅㅆㅇㅈㅉㅊㅋㅌㅍㅎ")
"char list: Hangul initials (초성)"



MEDIALS = list("ㅏㅐㅑㅒㅓㅔㅕㅖㅗㅘㅙㅚㅛㅜㅝㅞㅟㅠㅡㅢㅣ")
"char list: Hangul medials (중성)"



FINALS = list("∅ㄱㄲㄳㄴㄵㄶㄷㄹㄺㄻㄼㄽㄾㄿㅀㅁㅂㅄㅅㅆㅇㅈㅊㅋㅌㅍㅎ")
"char list: Hangul finals (종성)."



SPACE_TOKEN = " "
LABELS = sorted({SPACE_TOKEN}.union(INITIALS).union(MEDIALS).union(FINALS))

"char list: All CTC labels."





def check_syllable(char):
  return 0xAC00 <= ord(char) <= 0xD7A3



def split_syllable(char):
  assert check_syllable(char)
  diff = ord(char) - 0xAC00
  _m = diff % 28
  _d = (diff - _m) // 28
  return (INITIALS[_d // 21], MEDIALS[_d % 21], FINALS[_m])



def preprocess(str):
  result = ""
  for char in re.sub("\\s+", SPACE_TOKEN, str.strip()):
    if char == SPACE_TOKEN:
      result += SPACE_TOKEN
    elif check_syllable(char):
      result += "".join(split_syllable(char))
  return result


def join_syllable(char_list):
  num = 0xAC00 + 28 * 21 * (INITIALS.index(char_list[0])) + 28 * (MEDIALS.index(char_list[1])) + (FINALS.index(char_list[2]))
  s1 = chr(num)

In [ ]:
# 음성데이터 전처리 함수

import numpy as _np
from scipy.signal import resample_poly as _resample_poly
from python_speech_features import mfcc as _mfcc

SAMPLE_RATE = 16000

def mfcc(audio, samplerate, n_features, **kwargs):
  mfcc = _mfcc(audio, samplerate=samplerate, numcep=n_features, **kwargs)
  return mfcc

In [ ]:
# pcm 데이터 불러오기

# data_dir : pcm directory path
data_dir = "../"
# data_dir = "C:/Users/sooki/Desktop/data/001.일반남녀/001.일반남녀/성인남녀_001_A_001_M_KHI00_24_수도권_녹음실"

pcm_file = []
for i in os.listdir(data_dir):
    if 'PCM' in i:
        pcm_file.append(data_dir + "/" + i)
pcm_file[:3]

['C:/Users/sooki/Desktop/data/001.일반남녀/001.일반남녀/성인남녀_001_A_001_M_KHI00_24_수도권_녹음실/성인남녀_001_A_001_M_KHI00_24_수도권_녹음실_00001.PCM',
 'C:/Users/sooki/Desktop/data/001.일반남녀/001.일반남녀/성인남녀_001_A_001_M_KHI00_24_수도권_녹음실/성인남녀_001_A_001_M_KHI00_24_수도권_녹음실_00002.PCM',
 'C:/Users/sooki/Desktop/data/001.일반남녀/001.일반남녀/성인남녀_001_A_001_M_KHI00_24_수도권_녹음실/성인남녀_001_A_001_M_KHI00_24_수도권_녹음실_00003.PCM']

In [ ]:
## pcm -> wav 파일변환 

def pcm2wav(pcm_file, wav_file, channels=1, bit_depth=16, sampling_rate=16000):
    if bit_depth %8 != 0:
        raise ValueError("bit_depth"+str(bit_depth)+"must be a multiple of 8")
        
    with open(pcm_file, 'rb') as opened_pcm_file:
        pcm_data = opened_pcm_file.read()
        
        obj2write = wave.open(wav_file, 'wb')
        obj2write.setnchannels(channels)
        obj2write.setsampwidth(bit_depth//8)
        obj2write.setframerate(sampling_rate)
        obj2write.writeframes(pcm_data)
        obj2write.close
        
        
wav_file = [i.replace('PCM', 'wav') for i in pcm_file]
for pcm, wav in zip(sound_file, wav_file):
    pcm2wav(pcm, wav, 1, 16, 16000)

In [ ]:
# pcm2text 전사 텍스트 추출 리스트
labels = []

In [ ]:
# o : pcm2text 파일 경로
# pcm2text가 여러 가지 라면 경로를 바꿔가며 append 한다. 

o = open("../", "r")
# o = open("C:/Users/sooki/Desktop/data/002.노인남녀(시니어)/002.노인남녀(시니어)/000.PCM2TEXT/2020_시니어_학습DB_PCM2TEXT.list", 'r', encoding='cp949') 

k = o.readlines()
for i in k:
    index = i.index('PCM')
    string = i[index+4 : -1]
    labels.append(string)

In [ ]:
#텍스트 정제 (특수기호 제거)

for i, document in enumerate(labels):
    document = re.sub(r'[.,!?"\':;~()…]', '', document) #특수기호 제거, 정규 표현식
    labels[i] = document

In [ ]:
# 띄어쓰기, 맞춤법

a = '.'.join(labels)
take1 = spacing(a)
b = take1.split('.')
c = '.'.join(b)

spelled_sent = spell_checker.check(c)
take2 = spelled_sent.checked
take2=spell_checker.check(b)

len_t=int(len(take2))
text=[]
for i in range(0,len_t):
    text.append(take2[i][2])

In [ ]:
# 데이터 생성

def load_data(wav_file, text):
    # 오디오 데이터 전처리
    preprocess_audio = []
    for audio in wave_file:
        wav, sr = librosa.load(audio)
        m = mfcc(wav, samplerate=sr, n_features=13)
        pre_audio.append(m)
    
    # 텍스트 전처리
    pre_text = []
    for char in text:
        pre = preprocess(char)
        pre_text.append(pre)
    
    return (pre_audiio, pre_text)


data = load_data(wav_file, text)
data = list(filter(lambda x: x[0] is not None,data))

In [ ]:
# 하이퍼 파라메터 설정

params = tf.contrib.training.HParams(
    learning_rate=0.002,
    n_classes=54,
    train_steps=10,
    min_eval_frequency=100,
    use_fp16=False,
    num_filters=5,
    temporal_stride=1,
    keep_prob=0.5,
    batch_size=100,
    num_hidden=50,
    num_rnn_layers=3,
    rnn_type='uni-dir'
)


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [ ]:
# 텍스트 인코딩/디코딩 함수

NUM_CLASSES = 54
feat_len = 13
n_features = 50

ctc_labels = LABELS
labels = [" "] + ctc_labels
labels

_encode_map = {c: i for i, c in enumerate(labels)}
_decode_map = labels + [""]


def encode(texts):
  indices = []
  values = []
  for index, text in enumerate(texts):
    indices.extend(zip([index] * len(text), range(len(text))))
    values.extend(map(lambda c: _encode_map[c], text))
  indices = np.asarray(indices, dtype=np.int64)
  values = np.asarray(values, dtype=np.int32)
  shape = np.asarray([len(texts), np.asarray(indices).max(0)[1] + 1],dtype=np.int64)
  return indices, values, shape

def decode(sequences):
  """Decode CTC output to texts."""
  return list(map(lambda s: "".join(map(lambda c: _decode_map[c], s)), sequences))


In [ ]:
# acoustic model (음향모델)

def acoustic_model(feats, seq_len, targets, params, is_train):
    with tf.variable_scope('conv1') as scope:

        
        output_dropout_prob=0.3
        state_dropout_prob=0.3
        output_keep_prob = tf.cond(is_train, lambda: 1.0-output_dropout_prob, lambda: 1.0)
        state_keep_prob = tf.cond(is_train, lambda: 1.0-state_dropout_prob, lambda: 1.0)
        
        kernel = tf.get_variable('weights',
            shape=[10, 1, 1, params.num_filters],initializer=\
            tf.contrib.layers.variance_scaling_initializer(factor=2.0,
                                                       mode='FAN_IN',
                                                       uniform=False,
                                                       seed=None,
                                                       dtype= tf.float32), dtype=tf.float32)

        feats = tf.expand_dims(feats, dim=-1)
        conv = tf.nn.conv2d(feats, kernel,
                            [1, params.temporal_stride, 1, 1],padding='SAME')
        # conv = tf.nn.atrous_conv2d(feats, kernel, rate=2, padding='SAME')
        biases = tf.get_variable('biases', [params.num_filters],
                                  initializer=tf.constant_initializer(0),
                                  dtype=tf.float32)
        bias = tf.nn.bias_add(conv, biases)
        conv1 = tf.nn.relu(bias, name=scope.name)
        conv1_drop = tf.nn.dropout(conv1, output_keep_prob)

    # recurrent layers
    with tf.variable_scope('rnn') as scope:
        
        rnn_input = tf.reshape(conv1_drop, [params.batch_size, -1,feat_len*params.num_filters])

        
        
        cells = [tf.contrib.rnn.LSTMCell(100) for _ in range(2)]
        cells = [tf.contrib.rnn.DropoutWrapper(cell,
                                           output_keep_prob=output_keep_prob,
                                           state_keep_prob=state_keep_prob,
                                           variational_recurrent=True,
                                           dtype=tf.float32)
             for cell in cells]
        outputs, _ = tf.nn.bidirectional_dynamic_rnn(*cells, rnn_input, seq_len, dtype=tf.float32)
        outputs = tf.reshape(tf.concat(outputs, 2), [-1, 2 * 100])
        
        
        # Fully connected layer
        W = tf.Variable(tf.truncated_normal([2 * 100, params.n_classes], stddev=0.1))
        b = tf.Variable(tf.constant(0., shape=[params.n_classes]))
        logits = tf.transpose(tf.reshape(tf.matmul(outputs, W) + b, [tf.shape(feats)[0], -1, params.n_classes]), (1, 0, 2))     

        decoded,_=tf.nn.ctc_beam_search_decoder(logits, seq_len, beam_width=100)
        with tf.name_scope("Loss"):
            ctc = tf.reduce_mean(tf.nn.ctc_loss(targets,logits,seq_len,ignore_longer_outputs_than_inputs=True))
        
        with tf.name_scope("ler"):
            ler=tf.reduce_mean(tf.edit_distance(tf.cast(decoded[0], tf.int32), targets))
        with tf.name_scope("Decoded"):
            decoded=tf.sparse_tensor_to_dense(decoded[0],-1)
        


        return ctc, decoded, ler

In [ ]:
def feed(data, istrain):
    t1 = data[0]
    t2 = data[1]

    text = encode(t2)
       
    input_len_feed = np.asarray(list(map(len, t1)), np.int32)
    inputs_feed = np.zeros((len(t1),
                             max(input_len_feed),
                             feat_len),
                            np.float32)
    for i, l in enumerate(input_len_feed):
        inputs_feed[i, :l, :] = t1[i]
    return {inputs: inputs_feed, 
            input_len: input_len_feed,
            output: text,
            is_train:istrain}

In [ ]:
def batch_generator(data, batch_size, idx=0, num_workers=1, bin_size=100):
  bin_num = len(data[0]) // 100
  tmp = []
  bin_idx = 0
  i = data[0]
  j = data[1]
  for _ in range(bin_num):
    t0 = []
    for k, l in zip(i[bin_idx:bin_idx+bin_size], j[bin_idx:bin_idx+bin_size]): 
        t0.append([k, l])
        tmp.append(t0)
    bin_idx += bin_size
  order = list(range(0, len(i), batch_size))
  
  shuffle(order)
    
  data = sum(tmp, [])
  data_batch = []
  for i in order:
      if i % num_workers == idx:
           data_batch.append(list(zip(*data[i:(i+batch_size)])))
  return data_batch         
#

In [ ]:
_random.seed(29520)
        
n_train = int(len(data[0]) * 0.85/params.batch_size) * params.batch_size
# n_valid=n_train+int(len(data[0])*0.01)
n_valid = 100
# n_test=n_valid+int(len(data[0])*0.01)
n_test = 100

test_set=data[n_train+n_valid:n_train+n_valid+n_test]
valid_set=data[n_train+n_valid:n_valid]
batchs = {
      "train": batch_generator((data[0][:n_train], data[1][:n_train]) ,params.batch_size),
      "valid": batch_generator((data[0][n_train:n_train+n_valid],data[1][n_train:n_train+n_valid]), params.batch_size),
      "test": batch_generator((data[0][800:900], data[1][800:900]), params.batch_size)
  }

In [ ]:
with tf.device(tf.train.replica_device_setter(worker_device="/gpu:0",
                                                 ps_device="/cpu:0",
                                                 ps_tasks=1)):


  # with tf.Graph().as_default():   
        tf.reset_default_graph () 
        inputs = tf.placeholder(tf.float32, [None, None,feat_len])
        input_len = tf.placeholder(tf.int32, [None])
        output = tf.sparse_placeholder(tf.int32)
        is_train = tf.placeholder(tf.bool)
        cost, deco, ler=acoustic_model(inputs, input_len, output, params, is_train)
 

        tf.summary.scalar("CTC", cost)
        tf.summary.scalar("LER", ler)
        summary = tf.summary.merge_all()
        global_step = tf.train.get_or_create_global_step()

        train_op = tf.train.AdamOptimizer(params.learning_rate).minimize(cost, global_step,name="train_op")

In [ ]:
# train data set 학습 / validation data set 검증

config = tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True),
                         allow_soft_placement=True,
                         log_device_placement=True)
model_path =  "../model"
writers = {phase: tf.summary.FileWriter(path.join("./logs", phase)) for phase in ["train_sen2", "valid_sen2","test_sen2"]}



with tf.Session(config=config) as sess:
  try:
      saver = tf.train.Saver(tf.all_variables(),max_to_keep=500)  
      saver.restore(sess, tf.train.latest_checkpoint("../check"))
      global_step=tf.train.get_global_step()
      train_op = tf.get_default_graph().get_tensor_by_name("train_op:0")
      step = sess.run(global_step)
  except ValueError:
      sess.run(tf.global_variables_initializer())
      step=0

  
  for i in range(10):
        cost_total=ler_total=0
        print("===== TRAINING =====")
        
        for j in range(int(n_train/params.batch_size)):
            print(j)
            step,trainsum,_ = sess.run([global_step,summary,train_op], feed_dict=feed(batchs["train"][j], True))
            writers["train_sen2"].add_summary(trainsum, step)
        for k in range(int(n_valid/params.batch_size)):
          print(k)
          vcost,vler = sess.run([cost,ler], feed_dict=feed(batchs["valid"][k], False))
          print(vcost)
          print(vler)
          
          saver.save(sess, model_path, global_step=i)   

In [ ]:
# test data set 평가

with tf.Session(config=config) as sess: 
    print("===== TESTING =====")
    for l in range(int(n_test/params.batch_size)):
        testsum,str_decoded,test_cost, test_ler=sess.run([summary, deco, cost, ler], feed_dict=feed(batchs["test"][l], False))
        writers["test_sen2"].add_summary(testsum)
        print("LER: %f" %vler)
        for ss in zip(senior_text[:100], decode(str_decoded)):
            print("    Original: %s" % ss[0])
            print("    Decoded:  %s" % ss[1])
 

    print("Run 'tensorboard --logdir=%s'"%writers["train_sen2"].get_logdir())

In [ ]:
# pcm 파일 이름 추출

# o : pcm2list 경로
o = open("../", "r")
# o = open("C:/Users/sooki/Desktop/data/002.노인남녀(시니어)/002.노인남녀(시니어)/000.PCM2TEXT/2020_시니어_학습DB_PCM2TEXT.list", 'r', encoding='cp949') 

k = o.readlines()
file = []
for i in k:
    l = i.split('\t')
    string = l[0]
    file.append(string)

In [ ]:
# 인식결과 txt 파일 생성
f = open('recognition.txt', 'w', newline = '') 
for i, j in zip(file, decoded(str_decoded)):
    f.write('{0}\t{1}\n'.format(i, j))
f.close()

In [ ]:
# 전사결과 txt 파일 생성
g = open('text.txt', 'w', newline = '') 
for i, j in zip(file, text):
    g.write('{0}\t{1}\n'.format(i, j))
g.close()